In [1]:
from function import *
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
json_file = open("model.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("model.h5")

/home/rupamay/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
colors = []
for i in range(0,20):
    colors.append((245,117,16))
print(len(colors))
def prob_viz(res, actions, input_frame, colors,threshold):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

20


In [3]:
# 1. New detection variables
sequence = []
sentence = []
accuracy=[]
predictions = []
threshold = 0.8 

In [5]:
x, y, w, h = 300, 100, 300, 300

cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)

        # Make detections
        cropframe=frame[y:y+h, x:x+w]
        # print(frame.shape)
        frame=cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        # frame=cv2.putText(frame,"Active Region",(75,25),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,255,2)
        image, results = mediapipe_detection(cropframe, hands)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        try: 
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                
            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 
                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                                accuracy.append(str(res[np.argmax(res)]*100))
                        else:
                            sentence.append(actions[np.argmax(res)])
                            accuracy.append(str(res[np.argmax(res)]*100)) 

                if len(sentence) > 1: 
                    sentence = sentence[-1:]
                    accuracy=accuracy[-1:]

                # Viz probabilities
                # frame = prob_viz(res, actions, frame, colors,threshold)
        except Exception as e:
            # print(e)
            pass
            
        cv2.rectangle(frame, (300,100), (600, 69), (245, 117, 16), -1)
        cv2.putText(frame,"Output: -"+' '.join(sentence)+''.join(accuracy), (310,90), 
                       cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

/home/rupamay/anaconda3/lib/python3.8/site-packages/numpy/lib/shape_base.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = asanyarray(a)


1/1 [==============================] - 0s 499ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [==============================] - 0s 15ms/step
A
1/1 [======================

1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 17ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [=======================

1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [=======================

1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [=======================

1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 15ms/step
B
1/1 [==============================] - 0s 16ms/step
B
1/1 [=======================